# PREPOCESADO PYSPARK

Barquero Gómez de la Venta, Carlos

González Carrasco, Paula

Pérez Alba, Marina

Pérez Ortiz, Gema

In [1]:
pip install pyspark

     |████████████████████████████████| 212.3MB 62kB/s 
     |████████████████████████████████| 204kB 16.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=ec462554c811035a544e0ff4b30c27f33e67d2b19232d816c7595eab449b1f56
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
from pyspark import SparkContext
try:
    sc = SparkContext('local', 'Pyspark demo')
except ValueError:
    print('SparkContext already exists!')
from pyspark.sql import SparkSession
try:
    spark = SparkSession.builder.appName('Recommendation_system').getOrCreate()
except ValueError:
    print('SparkSession already exists!')

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import pyspark.sql.functions as F

In [5]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import unix_timestamp

from pyspark.sql.functions import from_unixtime
import datetime
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.types import StringType,BooleanType,DateType, DoubleType, TimestampType

In [6]:
import warnings 
warnings.filterwarnings("ignore")

import pandas as pd
from pandas import datetime as dt

In [7]:
from pyspark.sql import SparkSession,types
from pyspark.sql.functions import unix_timestamp, pandas_udf, PandasUDFType
from pyspark.sql.types import TimestampType, StructType
from operator import attrgetter

In [8]:
import time
start = time.time()
spark_df1 = spark.read.csv("/content/drive/MyDrive/country_vaccinations.csv", header = True)
spark_df2 = spark.read.csv("/content/drive/MyDrive/covid19_country_population.csv", header = True)

spark_df2 = spark_df2.withColumnRenamed("CountryAlpha3Code", "iso_code")

spark_df1 = spark_df1.drop(spark_df1.daily_vaccinations_raw)

spark_df1=spark_df1.withColumn("date",unix_timestamp("date", 'yyyy-MM-dd').cast(TimestampType()))

spark_df1 = spark_df1.withColumn("total_vaccinations",spark_df1.total_vaccinations.cast(DoubleType()))
spark_df1 = spark_df1.withColumn("people_vaccinated",spark_df1.total_vaccinations.cast(DoubleType()))
spark_df1 = spark_df1.withColumn("people_fully_vaccinated",spark_df1.total_vaccinations.cast(DoubleType()))
spark_df1 = spark_df1.withColumn("daily_vaccinations",spark_df1.total_vaccinations.cast(DoubleType()))
spark_df1 = spark_df1.withColumn("total_vaccinations_per_hundred",spark_df1.total_vaccinations.cast(DoubleType()))
spark_df1 = spark_df1.withColumn("people_vaccinated_per_hundred",spark_df1.total_vaccinations.cast(DoubleType()))
spark_df1 = spark_df1.withColumn("people_fully_vaccinated_per_hundred",spark_df1.total_vaccinations.cast(DoubleType()))
spark_df1 = spark_df1.withColumn("daily_vaccinations_per_million",spark_df1.total_vaccinations.cast(DoubleType()))

newDF = spark_df1.join(spark_df2, spark_df1.iso_code == spark_df2.iso_code, 'inner').drop(spark_df2.iso_code)

@pandas_udf(newDF.schema, PandasUDFType.GROUPED_MAP)
def interpolate(pdf):
    pdf = pdf.set_index('date')
    #pdf.sort_values(by=['a'], inplace=True)
    pdf = pdf.interpolate(method='cubicspline', axis=0, limit_direction='forward')
    pdf.reset_index(inplace=True)
    return pdf

newDF = newDF.groupby(['country']).apply(interpolate)
end = time.time()
print(end - start)


8.16568899154663
